# Applications processing Automation

The purpose of this code is to automate the first trivial filtering steps in the processing of the applications for the TReND in Africa Computational Neuroscience and Machine Learning Basics course.

This code is organized as a set of functions to be applied as a processing pipeline on the application responses data (See [documentation](https://docs.google.com/document/d/1n4pMEOgMuenuFpN6zXQtZlpYFXwPat2P4-SzZaN8mFg/edit?usp=drivesdk)).

### **How to use (as a developer):**
Just clone the Github repository and get into the business!\
If you have anaconda and yupyter installed locally you can just clone the repory directly on your machine. Elsewise, you can clone it into Google Colab.
(In either case, don't forget to regularly pull and push changes).

### **How to use (as a reviewer):**
If you are on Github now, open this notebook in Google Colab, or clone the whole repo locally, so you can run the cells. In case of running it in Colab, don't forget to save and download the resulting Excel sheet of the processed responses into a local folder.

In [1]:
import numpy as np
import pandas as pd

In [15]:
# Note that you have to download the responses data Excel sheet from Google Drive and put it in the same folder as the code.
# You don't have to do this if you cloned the Github repo (all will be organized in the repo).

# TODO: Load data directly from Google Drive.

DATA_DIR = './Copy of Answers_Application_form_TReND_Comp_Neuro_FIRSTPASS.xlsx'

responses_df = pd.read_excel(DATA_DIR)

# Adding two columns to the responses DataFrame (initialized with None for all cells).
responses_df['flag'] = None  #String ("flagged" or None
responses_df['notes'] = None #String (Text of notes == reasaons for flagging)

responses_df.columns

Index(['Timestamp', 'Email address', 'First Name', 'Last Name',
       'Date of Birth', 'Nationality', 'Gender', 'Career Stage',
       'Name of current University or Research Institution',
       'Undergraduate degree (completed or ongoing, eg. Neuroscience, Mathematics)',
       'Master's degree (completed or ongoing, if applicable, eg. Neuroscience, Mathematics)',
       'PhD degree (completed or ongoing, if applicable, eg. Neuroscience, Mathematics)',
       'Current research focus or research focus of the last research project you were engaged in (if applicable)',
       'Why would you like to attend the course? (2000 characters max)',
       'How do you think you could contribute to the course?  (2000 characters max)',
       'At the end of the first week the students will start a short individual research project. What would be your dream project?  (2000 characters max)',
       'Please attach a 1-page CV in pdf format (documents longer than one page will be discarded). If you h

In [6]:
# Carefully specify names of the columns to be processed (mostly responses for essay questions).

# TODO: To avoid any naming mistakes, replace questions, as columns names, with a key (or simply, indexes).

essay_qs = [
    'Why would you like to attend the course? (2000 characters max)',
    'How do you think you could contribute to the course?  (2000 characters max)',
    'At the end of the first week the students will start a short individual research project. What would be your dream project?  (2000 characters max)'
]

In [11]:
# Specify the minimum and maximun number of words for ansewrs for essay questions.
# Note: These parameters apply for for all essay questions.

MIN_WORDS_NUM = 50
MAX_WORDS_NUM = 2000

## Utility functions

In [2]:
def word_count(answer):
    """
    Takes a specific answer (cell) of a specific essay question and returns the answer's number of words.
    """
    
    return len(answer.split())


def set_flag(responses_df, email):
    """
    Set the 'flag' column value to "flagged" for a response chosen by it's 'Email address'
    """
    
    # This modifies the DataFrame itself (i.e change in place)
    responses_df.loc[responses_df['Email address'] == email, 'flag'] = "flagged"

## Main Pipeline

In [3]:
def remove_duplicates(responses_df):
    """
    removes duplicated rows (responses) based on 'Email address' and keeps the last response submitted.
    Note: Some students may make changes to their responses and submit a new one,
    this's why this function keeps the last response submitted and removes preceding ones.
    
    TODO: Check with the organizers what else is an adequate action.
    
    params :
        response_df: the responses data (DataFrame)
    returns:
        response_df: An edited response_df with duplicates removed
    """
    
    responses_df.drop_duplicates(subset=['Email address'], keep='last')


def flag_duplicates(responses_df):
    """
    flags duplicated rows (responses) based on 'Email address' and keeps the last response submitted.
    Note: Some students may make changes to their responses and submit a new one,
    this's why this function keeps the last response submitted and removes preceding ones.
    
    TODO: Check with the organizers what else is an adequate action.
    
    params :
        response_df: the responses data (DataFrame)
    returns:
        responses_df: An edited responses_df with 'flag' column updated
    """
    
    responses_df['flag'], responses_df['notes'] = np.where(
        
        # True/False nupmy array (True: duplicated, False: unique)
        responses_df.duplicated(subset=['Email address'], keep='last'),
        
        # Update row values if the corresponding True/False array element is True
        ['flagged', responses_df['notes'] + " - " + "A duplicated response" + " - "],
        
        # Leave row values as they were if the corresponding True/False array element is False
        [response_df['flag'], responses_df['notes']]
    )


def flag_short(responses_df, essay_qs):
    """
    flags insufficently short answers (less than a specific lower limit) for a specified
    set of essay questions.
    
    params :
        response_df: the responses data (DataFrame)
        essay_qs   : essay questions (list)
    returns:
        response_df: An edited responses_df with short answers removed
    """
     
    # Go through all the responses and for each response go through the answers for the essay questions
    for index, response in responses_df.iterrows():
        for question in essay_qs:
            
            if response[question] < MIN_WORDS_NUM:
                responses['flag'] = "flagged"
                responses['notes'] = responses_df['notes'] + " - " + "Insufficient short answer/s" + " - "